In [ ]:
from libtiff import TIFF
import cv2
import numpy as np
import os 
import segmentation_models as sm
import natsort
import random
import imgaug.augmenters as iaa
import imgaug as ia
from keras.models import load_model
from tqdm import tqdm
from PIL import Image
import keras.backend as K
import time
import imageio
import configparser


In [ ]:
config = configparser.ConfigParser()
config.read('segmentation_inference.ini')


MODEL_WEIGHTS_PATH = config['model_params']['model_weights_path'] #Path to model weights
PATH = config['model_params']['images_path'] # Path to folder with test_images
DEST_PATH = config['model_params']['save_path'] #Path to save the converted tiff outputs from the model


In [ ]:
MODEL_WEIGHTS_PATH

In [ ]:
model = sm.Unet("resnet50",input_shape=(None,None,3),classes=5,activation="sigmoid",encoder_freeze = True)
model.load_weights(MODEL_WEIGHTS_PATH)

In [ ]:
def convert_numpy_to_tiff(tiff_npy, file_name, save_path):
    
    if not os.path.exists(save_path):
        os.mkdir(save_path)
        
    list_temp = []
    
    for i in range(0,5):
        tiff_npy[:,:,i][tiff_npy[:,:,i]==1]=255
        list_temp.append(tiff_npy[:,:,i])
        
    tiff = np.asarray(list_temp,dtype=np.uint8)
    imageio.mimwrite(os.path.join(save_path,os.path.splitext(file_name)[0]+".tif"),tiff)


In [ ]:
def perform_inference(model,path,dest_path):
    
    for files in tqdm(os.listdir(path)):
        list_buffer = []

        img = cv2.imread(os.path.join(PATH,files))
        shape = img.shape
        img = cv2.resize(img,(512,512))
        mask_pred = model.predict(np.expand_dims(img,axis=0))
        mask_pred = mask_pred[0]
        
        for i in range(0,5):
            mask_pred[:,:,i][mask_pred[:,:,i]>0.5]=1
            mask_pred[:,:,i][mask_pred[:,:,i]<=0.5]=0
            list_buffer+=[cv2.resize(mask_pred[:,:,i],(shape[1],shape[0]),interpolation=cv2.INTER_NEAREST)]
        
        img_mask = cv2.merge((list_buffer[0],list_buffer[1],list_buffer[2],list_buffer[3],list_buffer[4]))
        convert_numpy_to_tiff(img_mask,files,dest_path)

        

In [ ]:
perform_inference(model,PATH,DEST_PATH)
